In [1]:
import os
import re
import glob
import itertools
import numpy as np
import xarray as xr

xr.set_options(display_expand_data=False)

%config InlineBackend.figure_format='retina'

In [2]:
#TSIDs:
TSIDs = ['BESI', 'GREGOR', 'HOP', 'JIL', 'NICHTJIL', 'SOM', 
         'TEST-HOP', 'TEST-HOP-mgn', 'TEST-SOM-mgn', 'TEST-VST',
         'TS021', 'TS025', 'TS027', 'TS028', 'TS029', 'VST']

#Data File Path:
file_path = os.path.join(glob.glob('C:\\Users\\mvomstein\\projects\\vt-2pod')[0], 'DATA')

file_path_save = glob.glob(
    'C:\\Users\\mvomstein\\projects\\vt_2pod_data_analysis\\DATA_xr')[0]

In [11]:
survey = np.loadtxt(os.path.join(file_path,'survey_2023-01-05.tsv'),
                 delimiter="\t", dtype=str)

In [4]:
BSs = ['forearm', 'back-lower', 'back-lower-nsa', 'abdomen', 'thigh']
HVs = ['h', 'v']

bsDic = {'forearm': 0,
         'back-lower': 1,
         'back-lower-nsa': 2,
         'abdomen': 3,
         'thigh': 4}

hvDic = {'h': 0,
         'v': 1}

sexDic = {'F': 0, 'M': 1}

medicalDic = {0 : 'Keine',
              1 : 'Diabetes',
              2 : 'Kapaltunnel',
              3 : 'Neurodermitis',
              4 : 'Taktile Wahrnehmungsveraenderung',
              5 : 'ADHS',
              6 : 'Lernstoerung',
              7 : 'Erkrankung ZNS',
              8 : 'Beeintraechtigung Sehsinn'}

In [5]:
def getMatch(param, fpts, bs, hv, datatype):
    matches = [match for match in os.listdir(fpts) 
           if bs in match 
           and hv in match 
           and param in match
           and os.path.splitext(match)[1] == datatype
          ]
    #print(matches)
    if len(matches) >=2: raise Exception(f'MULTIPLE ENTRIES IN {TSID}: DATA NOT CLEAN', matches)
    return(matches)

In [6]:
xStr = ['postmean', 'poststd', 'stim', 'response', 'pGuess', 'pSlope', 'pThresh']
yVal = [np.arange(2.5, 61, 2.5),
        np.arange(2.5, 61, 2.5),
        np.arange(0, 50, 1),
        np.arange(0, 50, 1),
        np.linspace(0.01, 0.99, 100), #guess
        np.linspace(0.01, 10, 50), #slope
        np.linspace(0.01, 60, 31) #thresh
       ]
yStr = ['stimRange', 'stimRange', 'trial', 'trial', 'guessGrid', 'slopeGrid', 'threshGrid']

for i in range(0, len(xStr), 1):
    for TSID in TSIDs:
        fpTS = os.path.join(file_path, TSID)
        arr = np.random.normal(size=(len(yVal[i]),1,5,2))*np.nan
        for BS, HV in itertools.product(BSs, HVs):
            matches = getMatch('_'+xStr[i]+'_', fpTS, '_'+BS+'_', '_'+HV+'_', '.npy')
            if matches:
                x = re.split("_", matches[0])
                singleData =np.load(
                    os.path.join(fpTS, matches[0]), allow_pickle=True)
                arr[:len(singleData),0, bsDic[x[1]], hvDic[x[2]]] = singleData

                #x[0]: TSID, x[1]: bs, x[2]: hv

        da = xr.DataArray(
            arr,
            dims=[yStr[i], 'tsid', 'bs', 'hv'],
            coords={yStr[i]:yVal[i],
                    'tsid':[TSID], 'bs':BSs, 'hv':['h', 'v']},
            name=xStr[i]
        )
        da.to_netcdf(os.path.join(file_path_save, xStr[i], xStr[i]+TSID+'.nc'))

In [7]:
# eParams
# =============================================================================

yStr = ['eGuess', 'eLapse', 'eSlope', 'eThreshold', 
        'stdGuess', 'stdLapse', 'stdSlope', 'stdThreshold']

for TSID in TSIDs:
    fpTS = os.path.join(file_path, TSID)
    arr = np.random.normal(size=(8,1,5,2))*np.nan
    for BS, HV, i in itertools.product(BSs, HVs, range(0, len(yStr))):
        matches = getMatch('_'+yStr[i]+'_', fpTS, '_'+BS+'_', '_'+HV+'_', '.npy')
        if matches:
            x = re.split("_", matches[0])
            singleData =np.load(
                os.path.join(fpTS, matches[0]), allow_pickle=True)
            arr[i ,0, bsDic[x[1]], hvDic[x[2]]] = singleData

                #x[0]: TSID, x[1]: bs, x[2]: hv

    da = xr.DataArray(
        arr,
        dims=['variableEP', 'tsid', 'bs', 'hv'],
        coords={'variableEP':yStr,
                'tsid':[TSID], 'bs':BSs, 'hv':['h', 'v']},
        name='eParams'
    )
    da.to_netcdf(os.path.join(file_path_save, 'eParams', 'eParams'+TSID+'.nc'))

In [15]:
# tsMeta
# =============================================================================

yStr = ['sex', 'age', 'height', 'weight', 'medical', 'chonk', 'hair']

sex_missing = []
chonk_missing = []
for TSID in TSIDs:
    fpTS = os.path.join(file_path, TSID)
    arr = np.random.normal(size=(len(yStr),1,5,2))*np.nan
    
    # sex, age, height, weight, sickness
    # --------------------------------------------------------------------------
    if any(np.where(survey==TSID)[0]):
        sex_etc = survey[np.where(survey==TSID)[0][0]][3:8]
        arr[:5, 0, :, [0,1]] = [sexDic[sex_etc[0]],
                                int(sex_etc[1]), 
                                int(sex_etc[2]), 
                                int(sex_etc[3]),
                                int(sex_etc[4][-2])]
    else:
        sex_missing.append([TSID])
        
    # chonk, hair
    # --------------------------------------------------------------------------
    for BS, HV in itertools.product(BSs, HVs):
        matches = getMatch('_meta_', fpTS, '_'+BS+'_', '_'+HV+'_', '.txt')
        if matches:
            x = re.split("_", matches[0])
            chonk, hair = np.genfromtxt(fpTS+'/'+matches[0], dtype='str', skip_header=True)[1:,1]
            arr[-2:, 0, bsDic[x[1]], hvDic[x[2]]] = chonk, hair
            
        if not matches and getMatch('postmean', fpTS, '_'+BS+'_', '_'+HV+'_', '.npy'):
            chonk_missing.append([TSID, BS, HV]) #für manuelles nachtragen
    # --------------------------------------------------------------------------
        #x[0]: TSID, x[1]: bs, x[2]: hv

    da = xr.DataArray(
        arr,
        dims=['variableTSM', 'tsid', 'bs', 'hv'],
        coords={'variableTSM':yStr,
                'tsid':[TSID], 'bs':BSs, 'hv':['h', 'v']},
        name='tsMeta'
    )
    da.to_netcdf(os.path.join(file_path_save, 'tsMeta', 'tsMeta'+TSID+'.nc'))
    
if chonk_missing: raise Exception(f'CHONK AND HAIR MISSING FOR {chonk_missing}')
if sex_missing: raise Exception(f'TS META MISSING FOR {sex_missing}')

Exception: CHONK AND HAIR MISSING FOR [['GREGOR', 'forearm', 'h'], ['GREGOR', 'forearm', 'v'], ['JIL', 'forearm', 'h'], ['TEST-HOP', 'forearm', 'h'], ['TEST-HOP', 'back-lower', 'h'], ['TEST-HOP', 'back-lower', 'v'], ['TEST-HOP-mgn', 'back-lower', 'v'], ['TEST-HOP-mgn', 'back-lower-nsa', 'h'], ['TEST-SOM-mgn', 'back-lower', 'h'], ['TEST-SOM-mgn', 'back-lower', 'v'], ['TEST-VST', 'back-lower', 'h'], ['TEST-VST', 'back-lower', 'v'], ['TEST-VST', 'back-lower-nsa', 'h']]